In [1]:
import csv

def import_items(filepath):
    """
    Reads items from a CSV file.
    Assumes each row has at least one cell with an item name.
    """
    with open(filepath, newline='') as csvfile:
        reader = csv.reader(csvfile)
        return [row[0].strip() for row in reader if row and row[0].strip()]

def transaction_generator(items, start_offset, items_in_txn, txn_count):
    """
    A generator that yields transactions using a continuous sliding window.
    It starts at start_offset and picks items_in_txn consecutive items for each transaction,
    wrapping around if the end of the list is reached.
    """
    n = len(items)
    pointer = start_offset
    for _ in range(txn_count):
        txn = []
        # Get a sequence of items for this transaction
        for _ in range(items_in_txn):
            txn.append(items[pointer % n])
            pointer += 1
        yield txn

def write_transactions_csv(transactions, filename):
    """
    Writes transactions to a CSV file with a header.
    The transaction items are joined by a semicolon.
    """
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["TransactionID", "Items"])
        for idx, txn in enumerate(transactions, start=1):
            writer.writerow([idx, ';'.join(txn)])

def main():
    # Load items from the external CSV file.
    items = import_items('items.csv')
    if not items:
        print("Error: No items found in 'items.csv'.")
        return

    num_datasets = 5      # Number of different transaction files to create.
    transactions_per_db = 20
    items_per_txn = 3

    # Generate each dataset with a unique starting offset.
    for dataset in range(1, num_datasets + 1):
        # Use a unique offset for each dataset (e.g. dataset 1 starts at 0, dataset 2 at 3, etc.)
        offset = (dataset - 1) * items_per_txn
        transactions = list(transaction_generator(items, offset, items_per_txn, transactions_per_db))
        output_file = f"transactions_dataset_{dataset}.csv"
        write_transactions_csv(transactions, output_file)
        print(f"Dataset {dataset} saved to '{output_file}' with {transactions_per_db} transactions.")

if __name__ == '__main__':
    main()


Dataset 1 saved to 'transactions_dataset_1.csv' with 20 transactions.
Dataset 2 saved to 'transactions_dataset_2.csv' with 20 transactions.
Dataset 3 saved to 'transactions_dataset_3.csv' with 20 transactions.
Dataset 4 saved to 'transactions_dataset_4.csv' with 20 transactions.
Dataset 5 saved to 'transactions_dataset_5.csv' with 20 transactions.


In [3]:
import csv
from collections import defaultdict
from itertools import combinations

# ---------- FP-Tree Classes ----------

class FPNode:
    def __init__(self, item, parent):
        self.item = item          # Item name (None for root)
        self.count = 1            # Frequency count for this node
        self.parent = parent      # Reference to parent node
        self.children = {}        # Dictionary: item -> FPNode
        self.link = None          # Link to next node with the same item

    def increment(self, count=1):
        self.count += count

class FPTree:
    def __init__(self):
        self.root = FPNode(None, None)
        # Header table: item -> first node in linked list of FPNode's for that item
        self.header_table = {}

    def add_transaction(self, transaction):
        """Add a sorted transaction (list of items) to the FP-tree."""
        current_node = self.root
        for item in transaction:
            if item in current_node.children:
                current_node.children[item].increment()
            else:
                new_node = FPNode(item, current_node)
                current_node.children[item] = new_node
                # Update header table: chain nodes with the same item together.
                if item in self.header_table:
                    last_node = self.header_table[item]
                    while last_node.link is not None:
                        last_node = last_node.link
                    last_node.link = new_node
                else:
                    self.header_table[item] = new_node
            current_node = current_node.children[item]

# ---------- FP-Growth Functions ----------

def build_fp_tree(transactions, min_support):
    """
    Build an FP-tree given the list of transactions and a minimum support count.
    Transactions are lists of items.
    """
    # First pass: count item frequencies
    frequency = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            frequency[item] += 1

    # Keep only items meeting min_support
    freq_items = {item: count for item, count in frequency.items() if count >= min_support}
    if not freq_items:
        return None, None

    # Define a function to sort items in a transaction by frequency (and lexicographically for ties)
    def sort_items(transaction):
        filtered = [item for item in transaction if item in freq_items]
        return sorted(filtered, key=lambda i: (-freq_items[i], i))

    tree = FPTree()
    for transaction in transactions:
        sorted_transaction = sort_items(transaction)
        if sorted_transaction:
            tree.add_transaction(sorted_transaction)
    return tree, freq_items

def mine_fp_tree(tree, prefix, min_support, frequent_patterns):
    """
    Recursively mine the FP-tree to find all frequent patterns.
    'prefix' is a set of items leading to the current subtree.
    Updates frequent_patterns: dict mapping frozenset(pattern) -> support.
    """
    # Process each item in the header table
    for item in list(tree.header_table.keys()):
        new_pattern = prefix.copy()
        new_pattern.add(item)
        support = 0
        node = tree.header_table[item]
        while node is not None:
            support += node.count
            node = node.link
        if support >= min_support:
            frequent_patterns[frozenset(new_pattern)] = support

        # Build conditional pattern base for this item
        conditional_patterns = []
        node = tree.header_table[item]
        while node is not None:
            path = []
            parent = node.parent
            while parent is not None and parent.item is not None:
                path.append(parent.item)
                parent = parent.parent
            path = list(reversed(path))
            if path:
                # Include the path as many times as the node's count
                for _ in range(node.count):
                    conditional_patterns.append(path)
            node = node.link

        # Construct the conditional FP-tree and mine it recursively
        conditional_tree, _ = build_fp_tree(conditional_patterns, min_support)
        if conditional_tree is not None and conditional_tree.header_table:
            mine_fp_tree(conditional_tree, new_pattern, min_support, frequent_patterns)

def generate_association_rules(frequent_patterns, transactions, min_confidence):
    """
    Generate association rules from frequent patterns.
    Returns a list of tuples: (Antecedent, Consequent, support, confidence).
    """
    rules = []
    for itemset, itemset_support in frequent_patterns.items():
        if len(itemset) < 2:
            continue  # Cannot form a rule with less than two items
        # Consider every non-empty proper subset as the antecedent
        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                # Get support of antecedent (mine from frequent_patterns if possible)
                antecedent_support = frequent_patterns.get(antecedent)
                if antecedent_support is None:
                    # Fallback: compute support directly
                    antecedent_support = sum(1 for t in transactions if antecedent.issubset(t))
                confidence = itemset_support / antecedent_support
                if confidence >= min_confidence:
                    rules.append((antecedent, consequent, itemset_support, confidence))
    return rules

# ---------- Data Loading and Processing ----------

def read_transactions(file_path):
    """
    Reads transactions from a CSV file.
    Assumes the CSV has a header with columns "TransactionID" and "Items",
    where "Items" are semicolon-separated.
    Returns a list of transactions (each a set of items).
    """
    transactions = []
    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header
        for row in reader:
            if len(row) >= 2:
                items = [item.strip() for item in row[1].split(';') if item.strip()]
                transactions.append(set(items))
    return transactions

def process_transactions(file_path, min_support, min_confidence):
    transactions = read_transactions(file_path)
    print(f"Processing file: {file_path}")
    # Build FP-tree (transactions are converted to lists for ordering)
    transactions_list = [list(t) for t in transactions]
    tree, _ = build_fp_tree(transactions_list, min_support)
    if tree is None:
        print("No items meet the minimum support threshold.")
        return

    frequent_patterns = {}
    mine_fp_tree(tree, set(), min_support, frequent_patterns)
    print("Frequent Patterns (FP-Growth):")
    for pattern, support in frequent_patterns.items():
        print(f"  {set(pattern)}: support = {support}")

    rules = generate_association_rules(frequent_patterns, transactions, min_confidence)
    print("\nAssociation Rules:")
    for antecedent, consequent, support, confidence in rules:
        print(f"  {set(antecedent)} -> {set(consequent)} (support: {support}, confidence: {confidence:.2f})")
    print("-" * 40)

def main():
    # Set thresholds
    min_support = 2      # Minimum support count
    min_confidence = 0.5 # Minimum confidence threshold
    # Process multiple transaction files
    transaction_files = [f"transactions_dataset_{i}.csv" for i in range(1, 6)]
    for file_path in transaction_files:
        process_transactions(file_path, min_support, min_confidence)

if __name__ == "__main__":
    main()


Processing file: transactions_dataset_1.csv
Frequent Patterns (FP-Growth):
  {'Desk Top'}: support = 6
  {'Lab Top'}: support = 6
  {'Lab Top', 'Desk Top'}: support = 4
  {'External Hard-Drive', 'Lab Top'}: support = 2
  {'\ufeffDigital Camera'}: support = 6
  {'\ufeffDigital Camera', 'Lab Top'}: support = 4
  {'\ufeffDigital Camera', 'External Hard-Drive', 'Lab Top'}: support = 2
  {'\ufeffDigital Camera', 'Desk Top'}: support = 2
  {'\ufeffDigital Camera', 'Lab Top', 'Desk Top'}: support = 2
  {'\ufeffDigital Camera', 'External Hard-Drive'}: support = 4
  {'\ufeffDigital Camera', 'Anti-Virus'}: support = 2
  {'\ufeffDigital Camera', 'External Hard-Drive', 'Anti-Virus'}: support = 2
  {'Flash Drive'}: support = 6
  {'Flash Drive', 'Desk Top'}: support = 2
  {'Microsoft Office'}: support = 6
  {'Flash Drive', 'Microsoft Office'}: support = 4
  {'Lab Top Case', 'Microsoft Office'}: support = 2
  {'Printer'}: support = 6
  {'Flash Drive', 'Printer'}: support = 4
  {'Flash Drive', 'Printe

In [4]:
import csv
import itertools
import sys
import time

def read_transactions(file_path):
    """
    Read transactions from a CSV file.
    Assumes the CSV has a header with at least the columns:
    - "TransactionID"
    - "Items" (semicolon-separated list of items)

    Returns a list of sets, where each set contains the items in a transaction.
    """
    transactions = []
    try:
        with open(file_path, 'r', newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                items_str = row.get("Items", "")
                if items_str:
                    # Convert the semicolon-separated items into a set after stripping whitespace
                    items = {item.strip() for item in items_str.split(';') if item.strip()}
                    if items:
                        transactions.append(items)
    except Exception as e:
        print(f"Error reading file '{file_path}': {e}")
        sys.exit(1)
    return transactions

def build_vertical_representation(transactions):
    """
    Builds a vertical data representation:
    Returns a dictionary mapping each item to the set of transaction indices in which it appears.
    """
    vertical = {}
    for tid, trans in enumerate(transactions):
        for item in trans:
            vertical.setdefault(item, set()).add(tid)
    return vertical

def eclat(prefix, prefix_tids, items, vertical, min_support, freq_itemsets):
    """
    Recursively mines frequent itemsets using the Eclat algorithm.

    Parameters:
      prefix      : current itemset (as a set)
      prefix_tids : set of transaction ids that contain the prefix (None for initial call)
      items       : list of items to extend the prefix with (in sorted order)
      vertical    : vertical representation mapping item -> set of tids
      min_support : minimum support count (absolute number)
      freq_itemsets: dictionary to store frequent itemsets (frozenset) and their support counts
    """
    for i, item in enumerate(items):
        new_itemset = prefix | {item}
        # If prefix is empty, take item's tid set directly; otherwise, intersect
        new_tids = vertical[item] if prefix_tids is None else prefix_tids & vertical[item]
        if len(new_tids) >= min_support:
            freq_itemsets[frozenset(new_itemset)] = len(new_tids)
            # Only consider items after the current one to avoid duplicates
            remaining_items = items[i+1:]
            eclat(new_itemset, new_tids, remaining_items, vertical, min_support, freq_itemsets)

def mine_frequent_itemsets(transactions, min_support):
    """
    Mines all frequent itemsets from the given transactions using the Eclat algorithm.

    Returns a dictionary mapping each frequent itemset (as a frozenset) to its support count.
    """
    vertical = build_vertical_representation(transactions)
    freq_itemsets = {}
    # Process items in sorted order (for consistent recursion order)
    sorted_items = sorted(vertical.keys())
    for i, item in enumerate(sorted_items):
        tids = vertical[item]
        if len(tids) >= min_support:
            freq_itemsets[frozenset({item})] = len(tids)
            eclat({item}, tids, sorted_items[i+1:], vertical, min_support, freq_itemsets)
    return freq_itemsets

def generate_association_rules(freq_itemsets, transactions, min_confidence):
    """
    Generates association rules from the frequent itemsets.

    For each frequent itemset (with 2 or more items), it considers every non-empty proper
    subset as a possible antecedent and computes the confidence of the rule.

    Returns a list of tuples: (antecedent, consequent, support_count, confidence)
    """
    rules = []
    for itemset in freq_itemsets:
        if len(itemset) < 2:
            continue
        itemset_support = freq_itemsets[itemset]
        # Generate all non-empty proper subsets
        for i in range(1, len(itemset)):
            for antecedent in itertools.combinations(itemset, i):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                if not consequent:
                    continue
                # Use the stored support for the antecedent if available; else, compute it.
                antecedent_support = freq_itemsets.get(antecedent)
                if antecedent_support is None:
                    antecedent_support = sum(1 for t in transactions if antecedent.issubset(t))
                confidence = itemset_support / antecedent_support if antecedent_support > 0 else 0
                if confidence >= min_confidence:
                    rules.append((antecedent, consequent, itemset_support, confidence))
    return rules

def main():
    file_path = input("Enter the path to the transaction CSV file: ").strip()
    transactions = read_transactions(file_path)
    if not transactions:
        print("No transactions loaded. Please check the file format.")
        sys.exit(1)
    print(f"Loaded {len(transactions)} transactions.")

    try:
        support_frac = float(input("Enter minimum support as a fraction (e.g., 0.05): ").strip())
        min_confidence = float(input("Enter minimum confidence (e.g., 0.5): ").strip())
    except ValueError:
        print("Invalid input for support or confidence. Please enter numeric values.")
        sys.exit(1)

    min_support_count = max(1, int(len(transactions) * support_frac))
    print(f"Using minimum support count: {min_support_count}")

    start_time = time.perf_counter()
    freq_itemsets = mine_frequent_itemsets(transactions, min_support_count)
    rules = generate_association_rules(freq_itemsets, transactions, min_confidence)
    end_time = time.perf_counter()

    print("\nFrequent Itemsets Discovered (Eclat):")
    for itemset, support in freq_itemsets.items():
        print(f"  {set(itemset)}: support = {support}")

    print("\nAssociation Rules Derived:")
    for antecedent, consequent, support, confidence in rules:
        print(f"  {set(antecedent)} -> {set(consequent)} (support: {support}, confidence: {confidence:.2f})")

    print(f"\nEclat mining and rule generation completed in {end_time - start_time:.4f} seconds.")

if __name__ == "__main__":
    main()


Enter the path to the transaction CSV file: transactions_dataset_1.csv
Loaded 20 transactions.
Enter minimum support as a fraction (e.g., 0.05): 0.2
Enter minimum confidence (e.g., 0.5): 0.2
Using minimum support count: 4

Frequent Itemsets Discovered (Eclat):
  {'Anti-Virus'}: support = 6
  {'External Hard-Drive', 'Anti-Virus'}: support = 4
  {'Lab Top Case', 'Anti-Virus'}: support = 4
  {'Desk Top'}: support = 6
  {'Lab Top', 'Desk Top'}: support = 4
  {'Printer', 'Desk Top'}: support = 4
  {'External Hard-Drive'}: support = 6
  {'\ufeffDigital Camera', 'External Hard-Drive'}: support = 4
  {'Flash Drive'}: support = 6
  {'Flash Drive', 'Microsoft Office'}: support = 4
  {'Flash Drive', 'Printer'}: support = 4
  {'Lab Top'}: support = 6
  {'\ufeffDigital Camera', 'Lab Top'}: support = 4
  {'Lab Top Case'}: support = 6
  {'Lab Top Case', 'Speakers'}: support = 4
  {'Microsoft Office'}: support = 6
  {'Speakers', 'Microsoft Office'}: support = 4
  {'Printer'}: support = 6
  {'Speakers'